<a href="https://colab.research.google.com/github/hrutkabence/tutorials/blob/master/english/data_processing/lessonsdxf_python_eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working DXF files with Python



DXF files are the text data exchange format used in AutoCAD.
Their content is identical to DWG files, but unlike DWG files, it is documented and available to everyone (https://docs.fileformat.com/cad/dxf/).
So, the content of DXF can be interpreted and created using a single program.

The content of DXF files is broken down into chapters (HEADER, TABLES, ENTITIES, etc.). The data is contained in two lines, the first containing a code and the second containing a text or numeric value.
The code determines how the following data will be interpreted (e.g. 10 - x coordinate, 2 - name block name/layer name depending on context).
In the following small detail (ENTITIES block), we have written comments after the codes and values; these are not part of the DXF file.

```
0                            ; Start of a section/entity
LINE                         ; type of entity
5                            ; entitiy ID
8F4                          ; hexadecimal ID value
100
AcDbEntity
8                            ; layer
GEOD_KERITES_KAPU            ; layer name
6                            ; line type
Continuous                   ; continuous line type
62                           ; colour code
56
100
AcDbLine
39
0.
10                           ; starting point x coordinate
590408.86
20                           ; starting point y coordinate
167862.39
30                           ; starting point z coordinate
0.
11                           ; ending point x coordinate
590409.82
21                           ; ending point y coordinate
167861.88
31                           ; ending point z coordinate
0.
```

## Creating DXF file from a coordinate list

Let's create a DXF file based on the IDs and coordinates given in a CSV file!


Last lesson we used the following code snippet to read the contents of a csv file into a *pandas* data frame.

In [ ]:
import pandas as pd
names = ['id', 'east', 'north', 'elev']
fname = 'coo_list.csv'
df = pd.read_csv(fname, names=names)

Let's download a CSV file with coordinates and print the some of the first rows!

In [5]:
!wget -q https://raw.githubusercontent.com/OSGeoLabBp/tutorials/master/english/data_processing/lessons/code/coo_list.csv
!head coo_list.csv

﻿base_1,667568.278,271801.865,252.176
1,655738.628,257962.892,117.020
600,655738.779,257962.765,117.019
601,655739.312,257962.986,117.049
602,655921.110,257840.450,117.644
603,655922.395,257844.152,117.472
604,655925.717,257853.584,117.733
605,655928.146,257860.510,117.552
606,655930.936,257868.434,117.240
607,655933.516,257874.923,116.623


Let's read the CSV file into a *pandas* data frame with error handling. In the previous list, it can be seen that there is no header in the file.

In [6]:
import pandas as pd
names = ['id', 'east', 'north', 'elev']
fname = 'coo_list.csv'
try:
    df = pd.read_csv(fname, names=names)
except FileNotFoundError:
    print(f'Following file is not existing:{fname}')

In [7]:
df.head()

,id,east,north,elev
0,base_1,667568.278,271801.865,252.176
1,1,655738.628,257962.892,117.020
2,600,655738.779,257962.765,117.019
3,601,655739.312,257962.986,117.049
4,602,655921.110,257840.450,117.644


Let's create a DXF file and write the minimum necessary introductory information! The file operations can also be verified here.

In [8]:
try:
    f_dxf = open('coo_list.dxf', 'w')
    print('  0\nSECTION\n  2\nENTITIES', file=f_dxf)
except:
    print('Failed to write into the file.')

Let's go through the rows of the data frame one by one and write a point entity and a text entity (point number) into the DXF file!

In [9]:
layer_p = "points"
layer_t = "point_id"
for index, row in df.iterrows():
    print(f"  0\nPOINT\n  8\n{layer_p}\n 10\n{row['east']:.3f}\n 20\n{row['north']:.3f}\n 30\n{row['elev']:.3f}", file=f_dxf)
    print(f"  0\nTEXT\n  8\n{layer_t}\n 10\n{row['east']+0.3:.3f}\n 20\n{row['north']+0.3:.3f}\n 30\n{row['elev']:.3f}\n 40\n1\n  1\n{row['id']}", file=f_dxf)

Let's close the DXF file!

In [10]:
print("  0\nENDSEC\n  0\nEOF", file=f_dxf)
f_dxf.close()

Copy the above three blocks of code, read the coordinate list, open the output file and print the DXF file's content to your own machine and take the name of the input file from the command line using the **argv** list of the **sys** module. Prepare the program to get the coordinate list via pipeline (e.g. from filt.py).

The following code block does not run in the Colab Environment, command line parameters cannot be passed, the *exit()* function is not executed in error handling! Run this on your own device!

In [ ]:
""" Creating DXF file from coordinte list"""
from sys import argv, stderr, stdin, stdout
from os import path
import pandas as pd

# Checking of command line arguments
if len(argv) < 2:
    # Using of stdin/stdout
    fp = stdin
    f_dxf = stdout
else:
    try:
        fp = open(argv[1])
    except:
        print(f"{argv[1]} file cannot be found!")
        exit()
    dxf_name = path.splitext(argv[1])[0] + '.dxf'
    try:
        f_dxf = open(dxf_name, "w")
    except:
        print(f"{dxf_name} file cannot be created!")
        exit()

# Reading of coordinate lists
names = ['id', 'east', 'north', 'elev']

try:
    df = pd.read_csv(fp, names=names)
except:
    print(f'File is not existing:{argv[1]}', file=stderr)
    exit()

# Creating DXF file
print('  0\nSECTION\n  2\nENTITIES', file=f_dxf)
layer_p = "points"
layer_t = "point_id"
for index, row in df.iterrows():
    print(f"  0\nPOINT\n  8\n{layer_p}\n 10\n{row['east']:.3f}\n 20\n{row['north']:.3f}\n 30\n{row['elev']:.3f}", file=f_dxf)
    print(f"  0\nTEXT\n  8\n{layer_t}\n 10\n{row['east']+0.3:.3f}\n 20\n{row['north']+0.3:.3f}\n 30\n{row['elev']:.3f}\n 40\n1\n  1\n{row['id']}", file=f_dxf)
print("  0\nENDSEC\n  0\nEOF", file=f_dxf)
if f_dxf != stdout:
    f_dxf.close()

## Using of [ezdxf](https://https://ezdxf.readthedocs.io/en/stable/) Python module

In the example above, we have written the corresponding codes and values directly into the DXF file. A more manageable solution is to use a Python package, where you don't need to know the code; you need to call the appropriate functions and methods. *ezdxf* is such a Python package.

*ezdxf* is not part of the Python installer; you have to install it separately with *pip* (package installer for Python).

In [ ]:
!pip install -q ezdxf
import ezdxf
from ezdxf.gfxattribs import GfxAttribs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.1 MB/s eta 0:00:00


Let's rewrite *pnt2dxf.py* to create the DXF file using *ezdxf*.

In [ ]:
doc = ezdxf.new()
doc.layers.new("points")
pnt_attr = GfxAttribs(layer="points")
doc.layers.new("point_id")
txt_attr = GfxAttribs(layer="point_id")
msp = doc.modelspace()
for index, row in df.iterrows():
    msp.add_point((row["east"], row["north"], row["elev"]), dxfattribs=pnt_attr)
    msp.add_text(row["id"], height=1, dxfattribs=txt_attr).set_placement((row["east"]+0.3, row["north"]+0.3))
doc.saveas('coo_list.dxf')

Let's check the DXF file you created with *ezdxf*!

In [11]:
dxf_file = 'coo_list.dxf'
doc = ezdxf.readfile(dxf_file)  # open and load DXF
model_space = doc.modelspace()
entities = {}
# count entities
for entity in model_space:
    entity_type = entity.dxftype()
    entities[entity_type] = entities.get(entity_type, 0) + 1
for typ, count in entities.items():
    print(f"{typ:6s}: {count:6}")

POINT :    161
TEXT  :    161


Let's reverse the task, i.e., extract the positions of the point elements from a DXF file into a CSV file with a line number. Consider only points on a given layer. Let's create a separate function to retrieve and print the data! Use the DXF file created in the first example.

In [12]:
def pnt_to_csv(msp, layer, fname):
    # msp - model space of DXF
    # layer - points on this layer are sent to CSV file
    # name of CSV file
    with open(fname, 'w') as csv_file:
        id = 1
        for entity in model_space:
            if entity.dxftype() == "POINT" and entity.dxf.layer == layer:
                print(f"{id},{entity.dxf.location[0]:.3f},{entity.dxf.location[1]:.3f},{entity.dxf.location[2]:.3f}", file=csv_file)
                id += 1

In [13]:
dxf_file = 'coo_list.dxf'
doc = ezdxf.readfile(dxf_file)  # open and load DXF
model_space = doc.modelspace()
pnt_to_csv(model_space, "points", "pnts.csv")

## Tasks

* Complete the first example (creating a DXF file from a coordinate list) by specifying the output file's name - the input file's name should be used for the output file name - and the delimiter character used in the CSV file. Use command line arguments!
* Modify the first example by adding point style and size!
* Modify the example to handle possible errors, e.g., non-existent CSV file, unmanageable DXF file structure, missing point number or coordinate, etc.
* Extend the program implementing the writing of point coordinates to DXF file to get parameters from the command line, collect points from multiple layers, use ezdxf's *query* function to sort.
* Transcribe the point list to DXF file example by using ezdxf package.
* Explore Python programs in the DXF_util repository on GitHub (https://github.com/zsiki/dxf_utils/blob/master/python) and apply them!
* Create another program to extend the DXF_util repository!
